In [1]:
import os

In [2]:
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
os.environ["groq_api_key"] = os.getenv("groq_api_key")

In [4]:
from langchain_groq import ChatGroq
llm=ChatGroq(model="qwen-2.5-32b")

In [5]:
llm.invoke("hii")

AIMessage(content='Hello! How can I assist you today?', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 31, 'total_tokens': 41, 'completion_time': 0.026380398, 'prompt_time': 0.003898626, 'queue_time': 0.021436352000000002, 'total_time': 0.030279024}, 'model_name': 'qwen-2.5-32b', 'system_fingerprint': 'fp_7b9ecfcd5f', 'finish_reason': 'stop', 'logprobs': None}, id='run-3967f498-d764-491b-ab0d-41a66cc8d9e2-0', usage_metadata={'input_tokens': 31, 'output_tokens': 10, 'total_tokens': 41})

In [6]:
from langchain_community.document_loaders import PyPDFLoader

In [7]:
file_path = r"C:\Users\ROHIT\OneDrive\Desktop\PN\RohitVyavahare_DataScientist(AI_Engineer)_Resume.pdf"
loader = PyPDFLoader(file_path)
docs=loader.load()
len(docs)

2

In [8]:
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [9]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=50)
splits=text_splitter.split_documents(docs)

In [10]:
embeddings=HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

d:\Users\ROHIT\anaconda3\envs\genai\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [11]:
from langchain_qdrant import QdrantVectorStore
from qdrant_client import QdrantClient
from qdrant_client.http.models import Distance, VectorParams


In [12]:
client = QdrantClient(":memory:")

client.create_collection(
    collection_name="demo_collection",
    vectors_config=VectorParams(size=384, distance=Distance.COSINE),
)

True

In [13]:
vector_store = QdrantVectorStore(
    client=client,
    collection_name="demo_collection",
    embedding=embeddings
)

In [14]:
vector_store.add_documents(documents=splits)

['9469c2e680d44fca901c9ef25bddb0b6',
 'e3029820f29a441cacb5cfcc101cd67f',
 'a0e74f44acd645599c300a2343d69232',
 'e164cc9574594c3cb13bc0f12453379c',
 'd5ae6bbbd85b4c7f8430f7c1e376fa14',
 '940a2a3a7f7c4928a111602ec174a177',
 'cddb57834e254916950b75a8eb199e54']

In [17]:
retreiver=vector_store.as_retriever()
retreiver.invoke("skills")

[Document(metadata={'source': 'C:\\Users\\ROHIT\\OneDrive\\Desktop\\PN\\RohitVyavahare_DataScientist(AI_Engineer)_Resume.pdf', 'page': 0, '_id': 'e3029820f29a441cacb5cfcc101cd67f', '_collection_name': 'demo_collection'}, page_content='• Machine Learning: Strong knowledge of supervised and unsupervised learning algorithms, including SVMs, Logistic \nRegression, Linear Regression, and Clustering. Experienced with machine learning libraries such as Scikit-Learn. \n• Deep Learning: Hands-on experience with deep learning techniques including CNNs, RNNs, LSTMs, Encoder-\nDecoder architectures, Transformers, and BERT. Familiar with deep learning frameworks such as TensorFlow, \nKeras, and PyTorch. \n• Natural Language Processing (NLP): Skilled in applying NLP techniques to analyze and extract insights from text \ndata. Proficient in using NLP libraries such as NLTK and working with GPT models for text generation and \nunderstanding. \n• API Development: Proficient in building and deploying AP

In [18]:
results = vector_store.similarity_search(
    "skills", k=2
)
for res in results:
    print(f"* {res.page_content} [{res.metadata}]")

* • Machine Learning: Strong knowledge of supervised and unsupervised learning algorithms, including SVMs, Logistic 
Regression, Linear Regression, and Clustering. Experienced with machine learning libraries such as Scikit-Learn. 
• Deep Learning: Hands-on experience with deep learning techniques including CNNs, RNNs, LSTMs, Encoder-
Decoder architectures, Transformers, and BERT. Familiar with deep learning frameworks such as TensorFlow, 
Keras, and PyTorch. 
• Natural Language Processing (NLP): Skilled in applying NLP techniques to analyze and extract insights from text 
data. Proficient in using NLP libraries such as NLTK and working with GPT models for text generation and 
understanding. 
• API Development: Proficient in building and deploying APIs using FLASK & FastAPI 
• Generative AI (GenAI): Experienced with GPT, LLaMA, Gemma, Mistral, and Ollama models. Proficient in Hugging [{'source': 'C:\\Users\\ROHIT\\OneDrive\\Desktop\\PN\\RohitVyavahare_DataScientist(AI_Engineer)_Resume.p

In [23]:
from langchain_core.prompts import ChatPromptTemplate
system_prompt = (
                "You are an assistant for question-answering tasks. "
                "Use the following pieces of retrieved context to answer "
                "the question. If you don't know the answer, say that you "
                "don't know. Use three sentences maximum and keep the "
                "answer concise."
                "{context}")

In [29]:
prompt=ChatPromptTemplate.from_messages(
    [
    ("system", system_prompt),
    ("user" ,"{input}")
])

In [35]:
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain

In [41]:
qa_chain= create_stuff_documents_chain(llm,prompt)
chain=create_retrieval_chain(retreiver,qa_chain)
result=chain.invoke({"input": "who is rohit"})
result["answer"]

'Rohit Vyavahare is a Data Scientist/AI Engineer with expertise in machine learning, deep learning, NLP, and Generative AI. He holds a Bachelor of Engineering in Civil Engineering from Pune University and has certifications in data science and machine learning. Rohit is proficient in Python, SQL, and various AI frameworks and libraries.'

In [43]:
print(result["context"][0].metadata)

{'source': 'C:\\Users\\ROHIT\\OneDrive\\Desktop\\PN\\RohitVyavahare_DataScientist(AI_Engineer)_Resume.pdf', 'page': 0, '_id': '9469c2e680d44fca901c9ef25bddb0b6', '_collection_name': 'demo_collection'}
